In [127]:
import cv2
import numpy as np
import tensorflow as tf

In [128]:
# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="dhito2-openmv\\trained.tflite")
interpreter.allocate_tensors()

In [129]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_x:0', 'index': 0, 'shape': array([ 1, 32, 32,  1]), 'shape_signature': array([ 1, 32, 32,  1]), 'dtype': <class 'numpy.int8'>, 'quantization': (0.003921568859368563, -128), 'quantization_parameters': {'scales': array([0.00392157], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 14, 'shape': array([1, 2]), 'shape_signature': array([1, 2]), 'dtype': <class 'numpy.int8'>, 'quantization': (0.00390625, -128), 'quantization_parameters': {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128]), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [130]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['x'], 'outputs': ['output_0']}}

In [131]:
# Load labels
with open("dhito2-openmv\labels.txt", "r") as f:
    labels = [line.strip() for line in f.readlines()]

In [132]:
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # print(f"Original image shape: {image.shape}")
    
    # Resize to 32x32
    resized = cv2.resize(image, (32, 32))
    # print(f"Resized image shape: {resized.shape}")
    
    # Normalize the image
    normalized = resized / 255.0
    # print(f"Normalized image: {normalized}")
    
    # Scale and quantize the image to int8
    input_scale, input_zero_point = input_details[0]['quantization']

    input_data = (normalized / input_scale + input_zero_point).astype(np.int8)
    # print(f"Quantized input data: {input_data}")
    
    return image, resized, input_data

In [133]:
def classify_image(image_path):
    original_image, resized_image, input_data = preprocess_image(image_path)
    
    # Add batch dimension to input data
    input_data = np.expand_dims(input_data, axis=0)
    input_data = np.expand_dims(input_data, axis=-1)

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()


    output_data = interpreter.get_tensor(output_details[0]['index'])
    # print(f"Raw output data: {output_data}")
    output_scale, output_zero_point = output_details[0]['quantization']
    # print(f"Output scale: {output_scale}, Output zero point: {output_zero_point}")
    predictions = (output_data.astype(np.float32) - output_zero_point) * output_scale
    # print(f"Dequantized output data: {predictions}")

    # Display the results
    for i, prob in enumerate(predictions[0]):
        print(f"{labels[i]}: {prob:.4f}")


In [134]:
original_image, resized_image, input_data = preprocess_image('img\matatutup.jpg')

In [135]:
cv2.imshow('Original Image', original_image)
cv2.imshow('Resized Image', resized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [136]:

classify_image('img\matatutup.jpg')

buka: 0.1641
tutup: 0.8359
